In [1]:
!pip install BeautifulSoup4

In [2]:
import pickle
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
# path
AIEEE = '../../../jalend/aieeedata/'
AIEEE_2004 = AIEEE+'2004.p'
AIEEE_2005 = AIEEE+'2005.p'
AIEEE_2006 = AIEEE+'2006.p'
AIEEE_2007 = AIEEE+'2007.p'
AIEEE_2008 = AIEEE+'2008.p'

In [4]:
with (open(AIEEE_2005, "rb")) as openfile:
    ai_09 = pickle.load(openfile)

In [5]:
rolls = ai_09.keys()

In [6]:
rn = []
for key in rolls:
    rn.append(key)

In [7]:
# Paper 1: Physics, Chemistry &amp; Mathematics
# Paper 2: Mathematics, Aptitude Test &amp; Drawing

In [14]:
data = []
count = 0
for elem in rn:
    student = []
    count = count+1
    if count%10000 == 0:
        print(count)
        df = pd.DataFrame(data, columns=['roll_no','name','mother_name','father_name','state','present','p1_marks','p2_marks','p3_marks','air_btech','state_btech','air_barch','state_barch'])
    if ai_09[elem] == "":
        continue
    soup = BeautifulSoup(ai_09[elem],'html.parser')
    tab  = soup.find_all('table', align="center", border="0", width="500")
    info0 = tab[0].find_all('td', align="left")
    roll_no = info0[0].string
    name = info0[1].string
    mother_name = info0[2].string
    father_name = info0[3].string
    student= student+([roll_no, name, mother_name, father_name])
    taba = soup.find_all('table',width='400')
#     try: 
    if len(tab) <= 2:
        present = False
        student.append(present)
#         print(taba[0].text,":",elem)
    else :
        present = True
        info2 = tab[1].find_all('td', align='right')
        p1_marks = info2[1].string
        p2_marks = info2[2].string
        p3_marks = info2[3].string
        state = tab[2].find_all('td')[-3].text
        info = tab[2].find_all('td', align='center')
        air_btech = info[2].text
        air_barch = info[3].text
        try:
            state_btech = info[4].text
            state_barch = info[5].text
            student=student+([state, present, p1_marks, p2_marks, p3_marks, air_btech, state_btech, air_barch, state_barch])
        except:
            print(roll_no)
            student=student+([state, present, p1_marks, p2_marks, p3_marks, air_btech, air_barch])

        
#     except:
#         import pdb; pdb.set_trace()
    data.append(student)
    

10000
 21700310 
20000
30000
40000


In [15]:
df = pd.DataFrame(data, columns=['roll_no','name','mother_name','father_name','state_no','present','p1_marks','p2_marks','p3_marks','air_btech','state_btech','air_barch','state_barch'])
    

In [16]:
df.shape

(48682, 13)

In [17]:
df.to_csv('new_data/aieee_2005.csv')

In [18]:
df.head()

,roll_no,name,mother_name,father_name,state_no,present,p1_marks,p2_marks,p3_marks,air_btech,state_btech,air_barch,state_barch
0,21300535,PRASHANT APARAJEYA,ABHA SINGH,RAM ASHISH SINGH,State Rank\n( State code of eligibility : 05 ),True,178.0,\r\n\r\n 66.5\r\n\r\n,\r\n\r\n NA,19642 *,1244,------,------
1,26601212,ANISHIYA P,RAJAMMAL,PAULRAJ,State Rank\n( State code of eligibility : 31 ),True,38.0,\r\n\r\n -5.0\r\n\r\n,\r\n\r\n NA,252135,9371,------,------
2,21601262,KSHITIJ DUA,RENU DUA,VINOD DUA,State Rank\n( State code of eligibility : 10 ),True,138.5,\r\n\r\n 56.5\r\n\r\n,\r\n\r\n NA,35231,3053,------,------
3,21630456,RAVI BHARDWAJ,RAMWATI BHARDWAJ,KHAZAN SINGH BHARDWAJ,State Rank\n( State code of eligibility : 33 ),True,15.0,\r\n\r\n -8.5\r\n\r\n,\r\n\r\n NA,340595,60369,------,------
4,26109362,RISHA MEHTA,PAWAN MEHTA,CHANDER MEHTA,False,None,None,None,None,None,None,None,None


Add Labels

In [19]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
import tensorflow as tf

In [20]:
def vocab():
    alpha = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
    special = [' ']
    vocab = alpha+special+['END']

    return vocab
    
def getIndex(df, maxlen, file):
    voc = vocab()
    char_index = dict((c, i) for i, c in enumerate(voc))
    
    X = []
    next_name = False
    
    # Club all the whitespaces
    
    trunc_X = [str(i).lower()[0:maxlen] for i in df['name']]
    mask = []
    for i in range(len(trunc_X)):
        name = trunc_X[i]
        tmp = []
        for char in str(name):
            if char not in voc:
                with open(file,'a') as f:
                    f.write(name+'\n')
                print(name)
                next_name = True
                break
            else:
                tmp.append(char_index[char])
        if next_name:
            mask.append(False)
            next_name = False
            continue
        mask.append(True)
        for k in range(0,maxlen - len(str(name))):
            tmp.append(char_index["END"])
        X.append(tmp)
            
    return np.asarray(X), df[mask]

In [21]:
import re
import numpy as np

def preprocess_df(df):
    df['name'] = [str(name).lower() for name in df['name']]
    df['name'] = [str(name).strip() for name in df['name']]
    df['name'] = [re.sub('[^a-zA-Z]+', ' ', str(name)) for name in df['name']]
    df['name'] = [re.sub('[ \t\n]+',' ', str(name)) for name in df['name']]
    df['name'].replace('', np.nan, inplace=True)
    df = df[df['name']!='nan']
    df.dropna(subset=['name'],inplace=True)
    return df

In [22]:
def returnGenderPred(df, model):
    test_x, df = getIndex(df, 30, 'oov.txt')
    pred = model.predict(np.asarray(test_x), verbose = 1)
    y_pred = np.argmax(pred, axis =1)
    df['Gender'] = y_pred
    return df

def returnCastePred(df, model):
    df = preprocess_df(df)
    test_x, df = getIndex(df, 30, 'oov.txt')
    pred = model.predict(np.asarray(test_x), verbose = 1)
    y_pred = np.argmax(pred, axis =1)
    df['Caste'] = y_pred
    return df

In [23]:
from keras.models import load_model


def get_gender_model():
    filename = "/home/souvic/mounted/btp/vahini/Models/ERTrain/SavedModels/FinalSet/CharLSTM/ERModel/ERModel"
    model = load_model(filename)
    return model

def get_caste_model():
    filename = "/home/souvic/mounted/btp/vahini/Name2Demographics/Models/AIEEEData/CNNModel/SavedModel/CasteModel"
    model = load_model(filename)
    return model

In [24]:
gender_model = get_gender_model()
caste_model = get_caste_model()

2022-02-14 23:11:22.919717: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-02-14 23:11:22.919814: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: obelix
2022-02-14 23:11:22.919832: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: obelix
2022-02-14 23:11:22.919999: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.86.0
2022-02-14 23:11:22.920056: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.86.0
2022-02-14 23:11:22.920071: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.86.0
2022-02-14 23:11:22.928046: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in perf

In [25]:
df = pd.read_csv("new_data/aieee_2005.csv")
df = preprocess_df(df)
df = returnGenderPred(df, gender_model)
df = returnCastePred(df, caste_model)
del df['Unnamed: 0']


1522/1522 [==============================] - 17s 11ms/step


In [26]:
df.to_csv("new_data/aieee_2005.csv")